In [1]:
from sage.all import matrix  # testing
from sage.all import *
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from flatsurf import *
import os
import pwlf
import os
from surface_dynamics.all import *
from Library import *
from Library import Section
import math
from time import time
import copy
from scipy import integrate
import sympy as sym
from sympy import Symbol
from sympy import solve, lambdify
from sympy import Rational, sqrt
import traceback
import dill
import sys
import unittest
from surface_dynamics.all import Origami
from utils import load_arrays_from_file  # testing
from fractions import Fraction as frac

In [2]:
# number of squares for STS
n_squares = 7
# index to start at
index = 0

permutations = perms_list(n_squares)

perm = permutations[index]

# get a list of saddle connections
vec_file = "vecs" + str(n_squares) + "-" + str(index) + ".npy"
vecs0 = load_arrays_from_file(os.path.join("vecs", vec_file))

In [162]:
def poincare_details2(perm, vecs0, generators):
    M = mathematica
    # find the eigenvectors for each generator and make sure they are 1
    eigs = []
    eigenvecs = []
    for matrix in generators:
        a = matrix[0][0]
        b = matrix[0][1]
        c = matrix[1][0]
        d = matrix[1][1]

        matrix_string = f"A = {{{{{a}, {b}}}, {{{c}, {d}}}}}"
        A = M(matrix_string)
        eigenvalues = M('eigenvalues = Eigenvalues[A]')
        eig1 = int(M('eigenvalues[[1]]'))
        eig2 = int(M('eigenvalues[[2]]'))

        if eig1 == eig2:
            if eig1 == 1:
                eigs.append(eig1)
            else:
                raise ValueError("Eigenvalue not equal to 1")
        else:
            raise ValueError("Different eigenvalues")
        
        eigenvectors = M('eigenvectors = Eigenvectors[A]')
        scaledEigenvector = M('scaledEigenvectors = Map[LCM @@ Denominator[#]*# &, eigenvectors][[1]]')
        scaled_x = int(scaledEigenvector.sage()[0])
        scaled_y = int(scaledEigenvector.sage()[1])
        vec = np.array([[scaled_x], [scaled_y]])
        eigenvecs.append(vec)

    # find the magnitude, slope, x-direction, and y-direction of each eigenvector
    def get_magnitude_slope_sign(vectors):
        # Stack the list of 2D numpy arrays into a 2D array
        vectors = np.hstack(vectors)  # Concatenate into a 2D array
        
        magnitudes = np.linalg.norm(vectors, axis=0)
        
        # Make sure the output array for division is explicitly float
        slopes = np.divide(vectors[1], vectors[0], out=np.full_like(vectors[1], float('inf'), dtype=np.float64), where=vectors[0] != 0)
        
        x_signs = np.sign(vectors[0])
        y_signs = np.sign(vectors[1])
        
        return magnitudes, slopes, x_signs, y_signs

    eigen_mags, eigen_slopes, eigen_x_signs, eigen_y_signs = get_magnitude_slope_sign(eigenvecs)
    saddle_mags, saddle_slopes, saddle_x_signs, saddle_y_signs = get_magnitude_slope_sign(vecs0)

    # Find corresponding saddle vectors for eigenvectors
    saddle_vecs = []
    for i in range(len(eigenvecs)):
        slope_vec = eigen_slopes[i]
        x_sign_vec = eigen_x_signs[i]
        y_sign_vec = eigen_y_signs[i]

        # Identify matches by slope, x, and y direction
        slope_matches = (slope_vec == saddle_slopes)
        x_sign_matches = x_sign_vec == saddle_x_signs
        y_sign_matches = y_sign_vec == saddle_y_signs

        valid_saddles = np.where(slope_matches & x_sign_matches & y_sign_matches)[0]

        if len(valid_saddles) == 0:
            raise DetailsError(f"No saddle vec for eigenvector {eigenvecs[i]}")

        # Select saddle with smallest magnitude
        smallest_idx = valid_saddles[np.argmin(saddle_mags[valid_saddles])]
        saddle_vecs.append(vecs0[smallest_idx])

    saddle_vecs = np.array(saddle_vecs)
    
    # find c_inv and c
    Cs = []
    C0s = []
    alphas = []
    Ms = []
    Ss = []
    S0s = []
    Ss = []
    scales = []
    mults = []
    Js = []
    for matrix, saddle_vec, eigen_vec in zip(generators, saddle_vecs, eigenvecs):
        a = matrix[0][0]
        b = matrix[0][1]
        c = matrix[1][0]
        d = matrix[1][1]
    
        matrix_string = f"A = {{{{{a}, {b}}}, {{{c}, {d}}}}}"
        A = M(matrix_string)
    
        S0, J = M('{S0, J} = JordanDecomposition[A]')
        S = M('S = Inverse[S0]')

        # print(S0)
        # print(S)
        # print()
        # print("-----------------------")
    
        S_np = np.array(S.sage())
        a_ = frac(str(S_np[0][0]))
        b_ = frac(str(S_np[0][1]))
        c_ = frac(str(S_np[1][0]))
        d_ = frac(str(S_np[1][1]))
        S = np.array([[a_, b_], [c_, d_]], dtype=object)
    
        M_np = np.array(J.sage())
        a_1 = int(M_np[0][0])
        b_1 = int(M_np[0][1])
        c_1 = int(M_np[1][0])
        d_1 = int(M_np[1][1])
        J = np.array([[a_1, b_1], [c_1, d_1]])

        S0_np = np.array(S0.sage())
        a_2 = frac(str(S0_np[0][0]))
        b_2 = frac(str(S0_np[0][1]))
        c_2 = frac(str(S0_np[1][0]))
        d_2 = frac(str(S0_np[1][1]))
        S0 = np.array([[a_2, b_2], [c_2, d_2]], dtype=object)
    
        if a_1 != 1 or c_1 != 0 or d_1 != 1:
            raise ValueError(f"wrong J: {J}")

        detC = (a_2*d_2) - (b_2*c_2)
        factor = int(1)/sqrt(detC)
        c0 = (factor * S0)
        x_ = (c0@np.array([[1],[0]]))[0][0]
        y_ = (c0@np.array([[1],[0]]))[1][0]
        mult0 = saddle_vec[0][0]/x_
        mult1 = saddle_vec[1][0]/y_
        if x_ != 0:
            mult = mult0
        elif y_ != 0:
            mult = mult1
        else:
            raise ValueError("Both coordinates in saddle_vec are zero. Division by zero is not defined.")
        scale = np.array([[mult, 0], [0, int(1)/mult]], dtype=object)
        c0 = c0 @ scale
        c = np.array([[c0[1][1], -c0[0][1]], [-c0[1][0], c0[0][0]]], dtype=object)

        J_new = c@matrix@c0
        a_1 = int(J_new[0][0])
        b_1 = int(J_new[0][1])
        c_1 = int(J_new[1][0])
        d_1 = int(J_new[1][1])

        if a_1 != 1 or c_1 != 0 or d_1 != 1:
            raise ValueError(f"wrong J_new: {J_new}")
        
        Cs.append(c)
        C0s.append(c0)
        Ms.append(J_new)
        Js.append(J)
        S0s.append(S0)
        Ss.append(S)
        scales.append(scale)
        mults.append(mult)
        alphas.append(J_new[0][1])
                    
    return alphas, Cs, C0s, Ss, S0s, eigs, Ms, Js, generators, eigenvecs, saddle_vecs, scales, mults

In [163]:
for i in range(100):
    gs = generators(perm, vecs0)
    alphas, Cs, C0s, Ss, S0s, eigs, Ms, Js, gens, eigenvecs, saddle_vecs, scales, mults = poincare_details2(perm, vecs0, gs)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [209]:
i = 9
Cs[i]@gens[i]@C0s[i]

array([[1, 7],
       [0, 1]], dtype=object)

In [210]:
Cs[i]

array([[0, 1/35],
       [-35, -9]], dtype=object)

In [211]:
C0s[i]

array([[-9, -1/35],
       [35, 0]], dtype=object)

In [212]:
Ms[i], Js[i]

(array([[1, 7],
        [0, 1]], dtype=object),
 array([[1, 1],
        [0, 1]]))

In [213]:
saddle_vecs[i]

array([[-9],
       [35]])

In [128]:
gens[i]

[1 2]
[0 1]

In [129]:
Ms[i]

array([[1, 1],
       [0, 1]])

In [130]:
scales[i]

array([[sqrt(2)/2, 0],
       [0, sqrt(2)]], dtype=object)

In [131]:
mults[i]

sqrt(2)/2